# Objectif de ce notebook : exporter dans un fichier .csv la liste des identifiants présents dans les notices IdRef d'une liste de chercheurs

In [1]:
import requests
import xml.etree.ElementTree as ET
import csv

## Interroger le webservice idref2id et obtenir les identifiants chercheurs
Le webservice idref2id développé par l'ABES fournit les identifiants d’une dizaine de systèmes (à spécifier dans la requête) à partir d’un identifiant IdRef

In [2]:
def get_chercheur_ids(ppn):
    url = f"https://www.idref.fr/services/idref2id/{ppn}"
    response = requests.get(url)
    if response.status_code == 200:
        tree = ET.fromstring(response.content)
        result_dict = {'PPN': ppn, 'HAL': '', 'ORCID': '', 'VIAF': '', 'WIKIPEDIA': '', 'ISNI': '', 'BNF': '', 'WIKIDATA': ''}
        for query in tree.findall('.//query'):
            for result in query.findall('.//result'):
                source = result.find('source').text
                identifiant = result.find('identifiant').text
                if source in result_dict:
                    result_dict[source] = identifiant
        return result_dict
    else:
        return None

## Lire les identifiants Idref depuis un fichier .txt

In [3]:
with open('..\data\idref.txt', 'r') as file:
    idrefs = file.read().splitlines()
all_results = []

## Itérer sur chaque Idref pour récupérer les identifiants chercheurs correspondants

In [4]:
for idref in idrefs:
    result = get_chercheur_ids(idref)
    if result:
        all_results.append(result)


## Définir le nom des colonnes du fichier CSV

In [5]:
headers = ['PPN', 'HAL', 'ORCID', 'VIAF', 'WIKIPEDIA', 'ISNI', 'BNF', 'WIKIDATA']

# Écrire les résultats dans un fichier CSV

In [7]:
with open('..\data\chercheurs_identifiants.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(all_results)
print(f"Les données ont été écrites avec succès")

Les données ont été écrites avec succès
